In [1]:
import xarray as xr
import matplotlib.pyplot as plt
from utility import analysis_parameters as ap
import numpy as np
import dask
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()

colors = ap.COLORS

In [12]:
model="SAM"
region="GT"
var_name="clivi"

# Global tropical cloud ice


In [11]:
chunk_dict = {"time":2880//8, "lat":250, "lon":2000} # "grid_size":500000} 
var = ap.load_iwp(model, region, total=False, chunks=chunk_dict)
var

returned ice only SAM GT


<xarray.DataArray 'clivi' (time: 2880, lat: 1612, lon: 9216)>
dask.array<open_dataset-bb2adef0eed9fb746ef882e68a94caa3clivi, shape=(2880, 1612, 9216), dtype=float32, chunksize=(360, 250, 2000), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-30 ... 2020-02-28T23:45:00
  * lon      (lon) float32 0.0 0.03906 0.07812 0.1172 ... 359.9 359.9 360.0
  * lat      (lat) float32 -29.99 -29.96 -29.93 -29.89 ... 29.93 29.96 29.99
Attributes:
    standard_name:  atmosphere_mass_content_of_cloud_ice
    long_name:      Ice Path
    units:          kg/m2
    cell_methods:   time: mean
    origin:         IWP

In [5]:
1.e1

10.0

In [3]:
bins = np.logspace(-7,1,12)
bins

array([1.00000000e-07, 5.33669923e-07, 2.84803587e-06, 1.51991108e-05,
       8.11130831e-05, 4.32876128e-04, 2.31012970e-03, 1.23284674e-02,
       6.57933225e-02, 3.51119173e-01, 1.87381742e+00, 1.00000000e+01])

In [14]:
if var_name[:2]=="rl":
    bins = np.linspace(80,310,40)
elif var_name[:2]=="rs":
    bins = np.linspace(-5,1300,40)
elif var_name=="clivi":
    bins = np.logspace(-7,1,40) 


In [15]:
hist, edges = dask.array.histogram(var, bins=bins, range=[bins[0],bins[-1]]) # range=[1e-7, 10], [70,320], [0,1300]
counts = hist.compute()


[##                                      ] | 7% Completed | 306.07 ss


KeyboardInterrupt: 

In [ ]:
counts

In [ ]:
counts.to_netcdf(ap.STATS+"{}/{}_{}_{}_counts.nc".format(region, region, model, var_name))

In [ ]:
## Histogram ##
x = (edges[1:]+edges[:-1])/2 # np.linspace(-7,1)
plt.plot(x, counts, color=colors[model]) #(bins[-1]-bins[0])/40) # 
plt.xscale("log")
# plt.xlim([80,310])
if var_name[0]=="r":
    plt.xlabel(" (W/m$^2$)")
elif var_name[:3]=="cli":
    plt.xlabel("IWP (kg/m$^2$)")
plt.ylabel("Count")
plt.title(model+" "+var_name)
plt.savefig("../plots/"+model+"_"+var_name+"_ITCZ.png")
plt.show()


In [3]:
# ceres syn 1hm 2001 02 - 2022 02
import xarray as xr
ds = xr.open_dataset("/work/bb1153/b380883/TWP/CERES_SYN1deg-1HM_Terra-Aqua-MODIS_Ed4.1_Subset_200102-202202.nc")
olr = ds.adj_atmos_lw_up_all_toa_1hm
olr.groupby(olr.time.dt.year).mean().mean(dim=["lat","lon"]).to_netcdf("/work/bb1153/b380883/TWP/CERES_1HM_Feb-yearly-mean.nc")

In [24]:
# ceres syn 1hm 2000 08 01 - 2019 09 10
ds = xr.open_dataset("/work/bb1153/b380883/dyamond1/TWP/TWP_CERES_yearlymean_20000801-20190910.nc").olrtoa_all
ds.mean(axis=(1,2)).to_netcdf("/work/bb1153/b380883/dyamond1/TWP/CERES_1HM_Aug1-Sept10-yearly-mean.nc")
